<a href="https://colab.research.google.com/github/nandobb1411/MLP_Churn/blob/main/Churn_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

checar github para melhores explicações sobre o codigo.


In [ ]:
import tensorflow as tf
import json
from tensorflow.keras import datasets, layers, models, Sequential, optimizers, regularizers
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import MinMaxScaler

Dataset

In [ ]:
df = pd.read_csv('/content/sample_data/dataset.csv', delimiter=',')
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15647572,Greece,504,Spain,Male,34,0,54980.81,1,1,1,136909.88,0
1,15797692,Volkova,659,France,Female,33,7,89939.62,1,1,0,136540.09,0
2,15713559,Onyemauchechukwu,473,Germany,Female,32,5,146602.25,2,1,1,72946.95,0
3,15595067,Zhirov,637,Spain,Female,40,6,0.00,2,1,1,181610.60,0
4,15810167,Scott,657,Spain,Male,75,7,126273.95,1,0,1,91673.60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,15770214,Bryant,754,France,Female,27,7,0.00,2,1,0,144134.64,0
8996,15720134,Reynolds,709,Germany,Male,30,9,115479.48,2,1,1,134732.99,0
8997,15700549,Alvares,721,France,Male,54,5,0.00,2,1,1,4493.12,0
8998,15704081,Findlay,595,Germany,Male,30,9,130682.11,2,1,1,57862.88,0


Como as informações são strings, para conseguirmos preve-las vamos anexar cada classe a um int:
Spain:0
France:1
Germany:2

Male:0
Female:2

In [ ]:
def handle_string_type_columns(customer_dataset_df):
  customer_dataset_df.loc[customer_dataset_df['Geography']== 'Spain', 'Geography'] = 0
  customer_dataset_df.loc[customer_dataset_df['Geography']== 'France', 'Geography'] = 1
  customer_dataset_df.loc[customer_dataset_df['Geography']== 'Germany', 'Geography'] = 2
  customer_dataset_df.loc[customer_dataset_df['Gender']== 'Male', 'Gender'] = 0
  customer_dataset_df.loc[customer_dataset_df['Gender']== 'Female', 'Gender'] = 1
  return customer_dataset_df

In [ ]:
df= handle_string_type_columns(df)

agora não temos mais string em nosso dataset

In [ ]:
df

,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,15647572,Greece,504,0,0,34,0,54980.81,1,1,1,136909.88,0
1,15797692,Volkova,659,1,1,33,7,89939.62,1,1,0,136540.09,0
2,15713559,Onyemauchechukwu,473,2,1,32,5,146602.25,2,1,1,72946.95,0
3,15595067,Zhirov,637,0,1,40,6,0.00,2,1,1,181610.60,0
4,15810167,Scott,657,0,0,75,7,126273.95,1,0,1,91673.60,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
8995,15770214,Bryant,754,1,1,27,7,0.00,2,1,0,144134.64,0
8996,15720134,Reynolds,709,2,0,30,9,115479.48,2,1,1,134732.99,0
8997,15700549,Alvares,721,1,0,54,5,0.00,2,1,1,4493.12,0
8998,15704081,Findlay,595,2,0,30,9,130682.11,2,1,1,57862.88,0


nos livramos das colunas que não vamos usar para o treinamento

In [ ]:
df= df.drop(columns=['CustomerId', 'Surname'])

Agora vou aplicar MinMax nas colunas que tem grandes quantidades de numeros continuos, o EstimatedSalary e uma delas

In [ ]:
scaler = MinMaxScaler()

In [ ]:
df[['CreditScore']]= scaler.fit_transform(df[['CreditScore']])

In [ ]:
df[df.columns]= scaler.fit_transform(df[df.columns])

In [ ]:
df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0.308,0.0,0.0,0.216216,0.0,0.219136,0.000000,1.0,1.0,0.684433,0.0
1,0.618,0.5,1.0,0.202703,0.7,0.358471,0.000000,1.0,0.0,0.682583,0.0
2,0.246,1.0,1.0,0.189189,0.5,0.584310,0.333333,1.0,1.0,0.364462,0.0
3,0.574,0.0,1.0,0.297297,0.6,0.000000,0.333333,1.0,1.0,0.908046,0.0
4,0.614,0.0,0.0,0.770270,0.7,0.503288,0.000000,0.0,1.0,0.458141,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8995,0.808,0.5,1.0,0.121622,0.7,0.000000,0.333333,1.0,0.0,0.720574,0.0
8996,0.718,1.0,0.0,0.162162,0.9,0.460264,0.333333,1.0,1.0,0.673543,0.0
8997,0.742,0.5,0.0,0.486486,0.5,0.000000,0.333333,1.0,1.0,0.022026,0.0
8998,0.490,1.0,0.0,0.162162,0.9,0.520857,0.333333,1.0,1.0,0.289005,0.0


Agora vamos dividir o dataset para o treniamento, a divisão e feita em test, train e val

In [ ]:
Y_df= df["Exited"]

In [ ]:
X_df= df.drop(columns=['Exited'])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_df, Y_df, test_size=0.2, random_state=7)

Como o keras (suporte para redes neurais) não aceita o formato de CSV, teremos que transformar o dataset em um array do numpy (que seria o formato aceito)

In [ ]:
X_train_numpy= X_train.to_numpy()
y_train_numpy= y_train.to_numpy()
X_test_numpy= X_test.to_numpy()
y_test_numpy= y_test.to_numpy()

Essa conversão faz todos os valores ficarem como obj, transformar para float haha

In [ ]:
X_train_numpy = np.asarray(X_train_numpy).astype('float32')
y_train_numpy = np.asarray(y_train_numpy).astype('float32')

X_test_numpy = np.asarray(X_test_numpy).astype('float32')
y_test_numpy = np.asarray(y_test_numpy).astype('float32')

Construindo um modelo sequencial no tensor flow com o suporte do keras

Função que recebe as metricas precision e recall, faz um calculo que calcula o score pra validar se o numero esta bom ou não, jeito de ver se nosso modelo ta deboas

TREINAMENTO

In [ ]:
model= Sequential()

Construindo as layers

In [ ]:
BATCH_SIZE = 32
model.add(tf.keras.Input(shape=(10,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [ ]:
model.compile(optimizer= optimizers.Adam(lr=0.01),loss='binary_crossentropy',metrics=[tf.keras.metrics.BinaryAccuracy(threshold=0.5), tf.keras.metrics.BinaryCrossentropy(), tf.keras.metrics.Precision(thresholds=0.5), tf.keras.metrics.Recall(thresholds=0.5)] )


/usr/local/lib/python3.7/dist-packages/keras/optimizers/optimizer_v2/adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Enfim, a linha fit, TREINAR

In [ ]:
model.fit(x= X_train_numpy, y=y_train_numpy, batch_size=BATCH_SIZE, epochs=80, verbose=2, validation_data= (X_test_numpy,y_test_numpy))


Epoch 1/80
225/225 - 2s - loss: 0.4738 - binary_accuracy: 0.7904 - binary_crossentropy: 0.4738 - precision: 0.2000 - recall: 0.0075 - val_loss: 0.4353 - val_binary_accuracy: 0.8028 - val_binary_crossentropy: 0.4353 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - 2s/epoch - 11ms/step
Epoch 2/80
225/225 - 1s - loss: 0.4066 - binary_accuracy: 0.8160 - binary_crossentropy: 0.4066 - precision: 0.6951 - recall: 0.1822 - val_loss: 0.4074 - val_binary_accuracy: 0.8272 - val_binary_crossentropy: 0.4074 - val_precision: 0.5632 - val_recall: 0.5521 - 615ms/epoch - 3ms/step
Epoch 3/80
225/225 - 1s - loss: 0.3681 - binary_accuracy: 0.8478 - binary_crossentropy: 0.3681 - precision: 0.7468 - recall: 0.3896 - val_loss: 0.3664 - val_binary_accuracy: 0.8600 - val_binary_crossentropy: 0.3664 - val_precision: 0.8160 - val_recall: 0.3746 - 598ms/epoch - 3ms/step
Epoch 4/80
225/225 - 1s - loss: 0.3627 - binary_accuracy: 0.8499 - binary_crossentropy: 0.3627 - precision: 0.7503 - recall: 0.4011 - val_l

Dados do modelo

In [ ]:
loss, acc, binary_crossentropy, precision, recall= model.evaluate(X_test_numpy, y_test_numpy)
print("Test Accuracy: "+ str(acc))
print("Test Precision: " + str(precision))
print('Test Recall: '+ str(recall))

57/57 [==============================] - 0s 2ms/step - loss: 0.3687 - binary_accuracy: 0.8583 - binary_crossentropy: 0.3687 - precision: 0.7381 - recall: 0.4366
Test Accuracy: 0.8583333492279053
Test Precision: 0.738095223903656
Test Recall: 0.43661972880363464
Test F1: 0.5486725707361845


In [ ]:
from sklearn.metrics import classification_report
b= np.array( [0.7] )
print( np.round(b) )

y_predicted= model.predict(X_test_numpy)
y_predicted

rounded_y_predicted= np.round(y_predicted)
print( classification_report(y_test_numpy, rounded_y_predicted) ) 

[1.]
57/57 [==============================] - 0s 1ms/step
              precision    recall  f1-score   support

         0.0       0.87      0.96      0.92      1445
         1.0       0.74      0.44      0.55       355

    accuracy                           0.86      1800
   macro avg       0.81      0.70      0.73      1800
weighted avg       0.85      0.86      0.84      1800



In [ ]:
pt = np.round(y_predicted)


Array com as previsões.

In [ ]:
pt[1:600]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],